In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from openai import AzureOpenAI
from io import BytesIO
import os

In [2]:
main_page = "https://www.maxaro.nl"
df_prodDes = pd.DataFrame(columns=['Product', 'Description', 'URL'])

client = AzureOpenAI(
    api_key = "54a267e072934050a8df635e4f6da7b5",
    api_version = "2023-05-15",
    azure_endpoint = "https://maxbotai.openai.azure.com/"
    )

In [ ]:
def get_links(main_page):

    categories = get_category_links(main_page)
    formatted_categories = [format(x) for x in categories]

    selected_categories = st.multiselect('Select Categories', formatted_categories, key='cat_select')

    if selected_categories != st.session_state['selected_categories']:
        # New selection made, reset subcategories
        st.session_state['subcategories'] = {}
        st.session_state['selected_categories'] = selected_categories

    for category in selected_categories:
        if category not in st.session_state['subcategories']:
            category_url = main_page + '/' + category
            st.session_state['subcategories'][category] = get_sub_links(category_url)

    urls = []
    for category in selected_categories:
        subcategories = st.session_state['subcategories'].get(category, [])
        
        if subcategories:
            formatted_subcategories = [format(x) for x in subcategories]
            selected_subcategories = st.multiselect(f'Select Subcategories for {category}', formatted_subcategories, default=formatted_subcategories, key=f'sub_select_{category}')

            for sub_link in selected_subcategories:
                sub_url = main_page + '/' + category + '/' + sub_link
                urls.append(sub_url)
        else:
            category_url = main_page + '/' + category
            urls.append(category_url)

    if st.button("Start", key='start'):
        for url in urls:    
            get_product_links(url)